In [3]:
from vedo import *
import os
from ipyvtklink.viewer import ViewInteractiveWidget
import pykitti
import numpy as np
import tensorflow as tf
import time
import pickle

#limit GPU memory ------------------------------------------------
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  try:
    memlim = 4*1024
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
  except RuntimeError as e:
    print(e)
#-----------------------------------------------------------------

from tensorflow.math import sin, cos, tan
import tensorflow_probability as tfp
import sys
current = os.getcwd()
parent_directory = os.path.dirname(current)
sys.path.append(parent_directory)
from ICET_spherical import ICET
from utils import *
from metpy.calc import lat_lon_grid_deltas
from pioneer.das.api.platform import Platform
from scipy.spatial.transform import Rotation as R
from pioneer.das.api.egomotion.imu_egomotion_provider import IMUEgomotionProvider as emp 
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2
%autosave 180
%matplotlib notebook

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-04-25 23:19:47.454909: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 23:19:47.455592: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 23:19:47.455756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 23:19:47.455878: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Goal: Simultaneously Estimate Translation, Rotation, and Distortion

## $\mathbf{A} = [\hat{X}_{ij}, \hat{m}_{ij}] = 
\begin{bmatrix}
% x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
x, ~ y, ~ z, ~ \phi, ~ \theta, ~ \psi, ~ x^+, ~y^+, ~z^+, ~\phi^+, ~\theta^+, ~\psi^+ \\
\end{bmatrix}$

We can use newton-raphson to find A

<!-- ## $y_i = \mathbf{h}(y_j, \hat{X}_{ij}, \hat{m}_{ij}) + \mathbf{H}_m \delta m + \mathbf{H}_x \delta x + \text{H.O.T.}$ -->

## $y_i = \mathbf{h}(y_j, \hat{A}_{ij}) + \mathbf{H}_A \delta A + \text{H.O.T.}$

## $\mathbf{H}_A \in \mathbb{R}^{4N \times 12} $

### $\mathbf{H}_A = [H_X, H_m]$

# Run Demo

In [4]:
# load point cloud
# # no distortion
# old_cloud =  np.load("sample_data/paper_figures/case1/raw_frame_0.npy")
# # m_hat = np.array([0., 0., 0., 0., 0., 0.0]) #actual motion
# m_hat = np.array([-3., 0., 0., 0., 0., 0.1]) #test wrap around

# # movement in x
# old_cloud =  np.load("sample_data/paper_figures/case2/raw_frame_3.npy") 
# m_hat = np.array([3, 0, 0., 0., 0., 0])
# # m_hat = np.array([3, 0, 0., 0., 0., -0.2]) #FOR DEBUG-- deform just a little
# gt =  np.load("sample_data/paper_figures/case2/base_vel_2.npy")

# movement in x, y, & yaw
old_cloud =  np.load("sample_data/paper_figures/case3/raw_frame_1.npy") 
m_hat = np.array([3, -1, 0., 0., 0., -1])
# m_hat = np.array([3, -1, 0.1, 0.1, 0.05, -1]) #FOR DEBUG-- deform a little extra
gt =  np.load("sample_data/paper_figures/case3/base_vel_2.npy")
# print(gt) 

# period_lidar = 1
# t_scale = (2*np.pi)/(-m_hat[-1] + (2*np.pi/period_lidar))
# print(t_scale)
# m_hat = m_hat*t_scale
# # m_hat[-1] = m_hat[-1]*t_scaled
# print(m_hat)

#downsample
old_cloud = old_cloud[::5,:]

In [5]:
from remove_motion_basic import linear_correction_old as lc 

#apply ground truth distortion according to m_hat
new_cloud = lc(old_cloud, m_hat) 
#set ground truth transform between clouds
X_gt = np.array([0.5, 0.5, 0.03, 0.004, 0.05, 0.06])

trans = X_gt[:3]
rot = R_tf(X_gt[3:]).numpy()

new_cloud = (new_cloud @ rot) + trans

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(old_cloud, c = "#CB2314"))
disp.append(Points(new_cloud, c = "#3F5151"))
plt.show(disp, "raw point clouds")
ViewInteractiveWidget(plt.window)

/home/derm/anaconda3/envs/py39/lib/python3.9/site-packages/vtkmodules/util/numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

### Attempt to solve with basic Newton-Raphson

In [6]:
from linear_corrector import LC
pc1 = old_cloud
pc2  = new_cloud
m_hat0 = m_hat
dc = LC(cloud1 = pc1, cloud2 = pc2, fid = 50, niter = 100, draw = False, m_hat0 = m_hat0,  mnp = 25, RM = True)

2023-04-25 23:19:50.914415: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


~~~~~~~~~~~Iteration  0 ~~~~~~~~~~
took 0.017432689666748047 sec  to apply motion profile
took 0.005654573440551758 sec to get H
m_hat:  [ 2.87787819 -1.06453347 -0.0792706  -0.02603057 -0.05644891 -0.99891627]
~~~~~~~~~~~Iteration  1 ~~~~~~~~~~
took 0.019040346145629883 sec  to apply motion profile
took 0.0033006668090820312 sec to get H


2023-04-25 23:19:51.623793: I tensorflow/core/util/cuda_solvers.cc:179] Creating GpuSolver handles for stream 0x74cc6b0
2023-04-25 23:19:51.684483: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


m_hat:  [ 2.79201746 -1.01800776 -0.10985991 -0.04131591 -0.08506193 -0.99399829]
~~~~~~~~~~~Iteration  2 ~~~~~~~~~~
took 0.016748428344726562 sec  to apply motion profile
took 0.003387928009033203 sec to get H
m_hat:  [ 2.75630569 -1.03049731 -0.13513052 -0.05079795 -0.10508515 -0.98510873]
~~~~~~~~~~~Iteration  3 ~~~~~~~~~~
took 0.016641616821289062 sec  to apply motion profile
took 0.0017132759094238281 sec to get H
m_hat:  [ 2.7350018  -1.03237021 -0.15288584 -0.0576822  -0.12075617 -0.97410131]
~~~~~~~~~~~Iteration  4 ~~~~~~~~~~
took 0.016201496124267578 sec  to apply motion profile
took 0.0017235279083251953 sec to get H
m_hat:  [ 2.80080366 -1.05170739 -0.1647159  -0.06435965 -0.13560027 -0.96128017]
~~~~~~~~~~~Iteration  5 ~~~~~~~~~~
took 0.017863988876342773 sec  to apply motion profile
took 0.0032455921173095703 sec to get H
m_hat:  [ 2.71205258 -1.10576272 -0.18108015 -0.06807262 -0.14922689 -0.95477432]
~~~~~~~~~~~Iteration  6 ~~~~~~~~~~
took 0.016482830047607422 sec  to ap

m_hat:  [-1.35998762  1.85311675  0.14152831 -0.06961258 -0.16302343 -0.72360635]
~~~~~~~~~~~Iteration  43 ~~~~~~~~~~
took 0.01484537124633789 sec  to apply motion profile
took 0.002054452896118164 sec to get H
m_hat:  [-1.25341749  1.88349366  0.12219875 -0.06540928 -0.16058221 -0.70299011]
~~~~~~~~~~~Iteration  44 ~~~~~~~~~~
took 0.01622605323791504 sec  to apply motion profile
took 0.00307464599609375 sec to get H
m_hat:  [-1.21501756  1.68525624  0.10828459 -0.06283869 -0.15921196 -0.61908215]
~~~~~~~~~~~Iteration  45 ~~~~~~~~~~
took 0.01429605484008789 sec  to apply motion profile
took 0.003212451934814453 sec to get H
m_hat:  [-1.12457395  1.87149322  0.08786925 -0.05784541 -0.15566681 -0.58670759]
~~~~~~~~~~~Iteration  46 ~~~~~~~~~~
took 0.016510963439941406 sec  to apply motion profile
took 0.003298044204711914 sec to get H
m_hat:  [-0.63880455  1.94652045  0.07780379 -0.0549547  -0.15258628 -0.52101862]
~~~~~~~~~~~Iteration  47 ~~~~~~~~~~
took 0.01597905158996582 sec  to apply

took 0.014973878860473633 sec  to apply motion profile

 ---checking for moving objects---
took 0.0016624927520751953 sec to get H
m_hat:  [ 0.63584775  2.00934744 -0.13546653 -0.02055459 -0.10252835 -0.10310154]
~~~~~~~~~~~Iteration  80 ~~~~~~~~~~
took 0.015021800994873047 sec  to apply motion profile

 ---checking for moving objects---
took 0.001649618148803711 sec to get H
m_hat:  [ 0.63591862  2.00901294 -0.13579237 -0.0206744  -0.10276746 -0.10463327]
~~~~~~~~~~~Iteration  81 ~~~~~~~~~~
took 0.014841556549072266 sec  to apply motion profile

 ---checking for moving objects---
took 0.0016546249389648438 sec to get H
m_hat:  [ 0.63599277  2.0086627  -0.13605922 -0.02077057 -0.10283668 -0.10623723]
~~~~~~~~~~~Iteration  82 ~~~~~~~~~~
took 0.01944136619567871 sec  to apply motion profile

 ---checking for moving objects---
took 0.0016720294952392578 sec to get H
m_hat:  [ 0.63606125  2.00833988 -0.13625285 -0.02085891 -0.10284108 -0.10771789]
~~~~~~~~~~~Iteration  83 ~~~~~~~~~~
took 0

In [9]:
# print(max(residual)) #????

# print(y_i-y_j)

print(y_i - y_j_undistort) #WHY IS THIS ZERO??????

residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
            np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1))#.flatten()

# print(residual)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [8]:
A_hat = np.array([0., 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
skip = 1 #50
y_i = new_cloud[::skip] #baseline
y_j = old_cloud[::skip] #distorted cloud

plt = Plotter(N = 1, axes = 4, bg = (1, 1, 1), interactive = True)
disp=[]
disp.append(Points(y_i[:,:3], c = "#2c7c94 ", alpha = 1, r=3.5))

runlen = 3
for count in range(runlen):
    
    print("~~~~ iteration ", count, "~~~~~~~~~~~")
    print("A_hat = ", A_hat)
    
    #decompose A_hat into X_hat and m_hat
    X_hat = A_hat[:6]
    m_hat = A_hat[6:]
        
    #apply last estimate of distortion correction
    y_j_undistort = lc(y_j, m_hat)
    print(np.shape(y_j_undistort))

    #apply last rigid transform
    rot = R_tf(X_hat[3:]).numpy()
    trans = X_hat[:3]
    y_j_undistort = (new_cloud @ rot) + trans

    #get H = [H_X, H_m]
    #get jacobain of distortion correction function
    H_m = dc.get_H_m(y_j_undistort, m_hat) 
#     print("H_m:", np.shape(H_m), "\n", H_m[:10])
    
    #get jacobian of rigid transform function 
    H_x = jacobian_tf(tf.transpose(tf.convert_to_tensor(y_j_undistort, tf.float32)), tf.convert_to_tensor(X_hat[3:], tf.float32)) # shape = [num of corr * 3, 6]
    #need to append on a row of zeros since we are working with homogeneous coordinates!
    H_x = tf.reshape(H_x, (tf.shape(H_x)[0]//3, 3, 6)) # -> need shape [#corr//4, 4, 6]
#     print("before:", np.shape(H_x), "\n", H_x[0])
    H_x = tf.concat([H_x, tf.zeros([len(H_x),1,6])], axis = 1)
    H_x = tf.reshape(H_x, (-1, 6))
#     print("after:", np.shape(H_x), "\n", H_x[:10])
    H_x = H_x.numpy()
        
    #delta_m =  ((H^T*H)^-1)(H^T)(yi - yj*)
    residual = (np.append(y_i, np.ones([len(y_i),1]), axis = 1) -
                np.append(y_j_undistort, np.ones([len(y_i),1]), axis = 1)).flatten()
    print("residual", np.shape(residual), "\n", 1000*residual)
    
    H = np.append(H_x, H_m, axis = 1)
    print("H: \n", np.shape(H))
    
    delta_A = np.linalg.pinv(H.T @ H) @ H.T @ residual
    print("delta_A", delta_A)
    #augment rigid transform components
    A_hat[:6] +=   delta_A[:6]
    #augment distortion components
    A_hat[6:9] -= delta_A[6:9]
    A_hat[9:] += delta_A[9:] #TODO: figure out why angles are backwards...    
    
    #plot updated cloud2
#     color = [0.5 + count/(runlen*2), 1 - count/runlen, count/runlen]
    disp.append(Points(y_j_undistort[:,:3], c = "#a65852 "))#, alpha = (count+1)/(runlen+1), r=3.5))

    
plt.show(disp, "12 State Solution")
ViewInteractiveWidget(plt.window)

~~~~ iteration  0 ~~~~~~~~~~~
A_hat =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(16564, 3)
residual (66256,) 
 [0. 0. 0. ... 0. 0. 0.]
H: 
 (66256, 12)
delta_A [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
~~~~ iteration  1 ~~~~~~~~~~~
A_hat =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(16564, 3)
residual (66256,) 
 [0. 0. 0. ... 0. 0. 0.]
H: 
 (66256, 12)
delta_A [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
~~~~ iteration  2 ~~~~~~~~~~~
A_hat =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(16564, 3)
residual (66256,) 
 [0. 0. 0. ... 0. 0. 0.]
H: 
 (66256, 12)
delta_A [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


ViewInteractiveWidget(height=1043, layout=Layout(height='auto', width='100%'), width=1280)

In [ ]:
16564 * 4
